<a href="https://colab.research.google.com/github/syuan0525/project-team/blob/main/oilprice2line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息素材準備
handler執行方法設計
啟動應用


'''

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''

流程解析

'''


In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''


# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序app

建置handler與 line_bot_api
'''
import os

if not os.path.isdir("/content/material"):
    os.mkdir("./material")
# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("Xk8H4R083NV9pc4TXlel6C4/S7+eP2F2zU8IA8Ovnrjw0XhH2JPS4GraqvUKDhOA+3q+EHrSabdDHAxwupFrsSknmk3R5RITOw6zGVGwrog0THJJpp7tekxfRKtxuKC4iPcJpZ/p9Pfe0x+Bul2l9wdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("c6b3351c6478f32ba38b299af38294c3")

In [ ]:
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import prettytable

In [ ]:
url = "https://gas.goodlife.tw/"
req = Request(url,headers={
    "user-agent" : "Mozilla/5.0"
  })
response = urlopen(req)
html = BeautifulSoup(response)

for i in html.find_all("div",id="cpc"):
  s1 = i.find_all("li")[0].find("h3")
  s1.extract()
  s2 = i.find_all("li")[1].find("h3")
  s2.extract()
  s3 = i.find_all("li")[2].find("h3")
  s3.extract()

df = pd.DataFrame(columns=["公司","92油價(每公升)","95油價(每公升)","98油價(每公升)"])
table1 = prettytable.PrettyTable(["公司","92油價(每公升)","95油價(每公升)","98油價(每公升)"])

for i in html.find_all("div",id="cpc"):
  company = i.find("h2").text
  price92 = i.find_all("li")[0].text.replace("\n", "")
  price95 = i.find_all("li")[1].text.replace("\n", "")
  price98 = i.find_all("li")[2].text.replace("\n", "")
  s = pd.Series([company,price92,price95,price98],index=["公司","92油價(每公升)","95油價(每公升)","98油價(每公升)"])
  df = df.append(s,ignore_index=True)
  table1.add_row([company,price92,price95,price98])




import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import table
import matplotlib as mpl
from matplotlib.font_manager import fontManager

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')
plt.figure(figsize=(6,4))
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, df)  # where df is your data frame

plt.savefig('./material/mytable.png')

In [ ]:
!pip install pyimgur

In [ ]:
import pyimgur

CLIENT_ID = "f39377e6676d4f0"
PATH = "./material/mytable.png" #A Filepath to an image on your computer"
title = "asdsada"
im = pyimgur.Imgur(CLIENT_ID)
uploaded_image = im.upload_image(PATH, title=title)
#把圖片url設為全域變數不然handler內無法取到值
global url
url = uploaded_image.link

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    
    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
        
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

specific_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")

# analyze_truth_image_message=ImageSendMessage(
#       original_content_url='https://i.imgur.com/M8jnn9B.png',
#       preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
#       )
analyze_truth_image_message=ImageSendMessage(
  original_content_url=url,
  preview_image_url=url
  )


In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@123":specific_text_message,
  "@分析的本質": analyze_truth_image_message
}

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event, request):
    # if(event.message.text.find('@')!= -1):
    #     line_bot_api.reply_message(
    #     event.reply_token,
    #     template_message_dict.get(event.message.text)
    #     )
    # else:
    #     line_bot_api.reply_message(
    #     event.reply_token,
    #     TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質"))
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get("@分析的本質"))

In [ ]:
# 主程序運行
app.run()